---
## 📦 Krok 0: Import bibliotek

In [ ]:
# Importy systemowe
import os
import warnings
warnings.filterwarnings('ignore')

# Upewnij się że katalogi istnieją
os.makedirs("data", exist_ok=True)
os.makedirs("output", exist_ok=True)
os.makedirs("output/visualizations", exist_ok=True)

print("✅ Katalogi przygotowane")

---
## 🎯 Krok 1: Generowanie danych testowych (PCAP)

Generujemy realistyczny ruch sieciowy zawierający:
- Normalny ruch HTTP/HTTPS/DNS
- Scenariusze ataków (port scan, C2, ransomware, exfiltration)

In [ ]:
from simulate_traffic import create_demo_pcap_with_alerts, create_apt_scenario_pcap, create_normal_traffic_pcap

# Generowanie plików PCAP
create_demo_pcap_with_alerts("data/demo_traffic.pcap")
create_normal_traffic_pcap("data/normal_traffic.pcap")
create_apt_scenario_pcap("data/apt_attack.pcap")

print("\n✅ Pliki PCAP wygenerowane!")

---
## 📊 Krok 2: Analiza przepływów sieciowych

Wczytujemy plik PCAP i ekstrahujemy przepływy sieciowe (5-tuple).

In [ ]:
from flow_analyzer import FlowAnalyzer

# Wczytanie przepływów
analyzer = FlowAnalyzer("data/demo_traffic.pcap")
flows = analyzer.load_flows()
stats = analyzer.get_flow_statistics()

# Wyświetlenie statystyk
print("=" * 50)
print("📈 STATYSTYKI PRZEPŁYWÓW")
print("=" * 50)
print(f"Całkowita liczba przepływów: {stats['total_flows']}")
print(f"Unikalne IP źródłowe:        {stats['unique_src_ips']}")
print(f"Unikalne IP docelowe:        {stats['unique_dst_ips']}")
print(f"Całkowita liczba pakietów:   {stats['total_packets']:,}")
print(f"Całkowita liczba bajtów:     {stats['total_bytes']:,}")
print(f"Średnia pakietów/przepływ:   {stats['avg_packets_per_flow']:.2f}")

In [ ]:
# Podgląd danych
print("\n📋 Przykładowe przepływy:")
flows.head(10)

---
## 🚨 Krok 3: Detekcja zagrożeń

Uruchamiamy silnik detekcji z regułami:
- **Python rules**: Port scan, Large transfer, Suspicious ports, DNS tunneling
- **Sigma rules**: Format YAML zgodny ze standardem Sigma

In [ ]:
from detection_rules.detection_rules import create_default_detection_engine

# Inicjalizacja silnika detekcji
engine = create_default_detection_engine()

# Uruchomienie detekcji
alerts = engine.run_detection(flows)
alerts_df = engine.get_alerts_dataframe()

print("=" * 50)
print("🚨 WYNIKI DETEKCJI")
print("=" * 50)
print(f"Wykryto alertów: {len(alerts)}")

if not alerts_df.empty and 'severity' in alerts_df.columns:
    print("\nRozkład według severity:")
    print(alerts_df['severity'].value_counts().to_string())

In [ ]:
# Lista alertów
if not alerts_df.empty:
    print("\n📋 Lista wykrytych alertów:")
    display(alerts_df[['rule', 'severity', 'message']].head(15))
else:
    print("Brak alertów")

---
## 🤖 Krok 4: Klasyfikacja Machine Learning

Trenujemy model ML do klasyfikacji przepływów jako normalne/podejrzane.

In [ ]:
from models.ml_classifier import NetworkMLClassifier

# Inicjalizacja i trenowanie modelu
classifier = NetworkMLClassifier()
ml_metrics = classifier.train(flows, tune_hyperparameters=False)

print("=" * 50)
print("🤖 METRYKI MODELU ML")
print("=" * 50)
print(f"Accuracy:  {ml_metrics['accuracy']:.4f}  ({ml_metrics['accuracy']:.2%})")
print(f"Precision: {ml_metrics['precision']:.4f}")
print(f"Recall:    {ml_metrics['recall']:.4f}")
print(f"F1 Score:  {ml_metrics['f1_score']:.4f}")
print(f"\n⚠️  FPR (False Positive Rate): {ml_metrics['fpr']:.4f}")
print(f"✅ TPR (True Positive Rate):  {ml_metrics['tpr']:.4f}")

In [ ]:
# Macierz konfuzji
import numpy as np

cm = ml_metrics['confusion_matrix']
print("\n📊 Macierz konfuzji:")
print(f"                  Predicted")
print(f"                  Normal  Suspicious")
print(f"Actual Normal     {cm[0][0]:5d}   {cm[0][1]:5d}")
print(f"       Suspicious {cm[1][0]:5d}   {cm[1][1]:5d}")

---
## 🌍 Krok 5: Threat Intelligence Enrichment

Wzbogacamy przepływy o dane geolokalizacyjne (kraj, miasto, ISP).

In [ ]:
from threat_intel import ThreatIntelligenceEnricher

# Inicjalizacja enrichera
enricher = ThreatIntelligenceEnricher(rate_limit_delay=0.3)

# Wzbogacenie danych (tylko publiczne IP docelowe)
print("🌍 Wzbogacanie danych o geolokalizację...")
print("(To może potrwać chwilę ze względu na limity API)\n")

enriched_flows = enricher.enrich_flows(
    flows, 
    enrich_src=False,  # Pomijamy prywatne IP źródłowe
    enrich_dst=True    # Wzbogacamy publiczne IP docelowe
)

print("\n✅ Wzbogacanie zakończone!")

In [ ]:
# Podgląd wzbogaconych danych
geo_columns = ['dst_ip', 'dst_country', 'dst_city', 'dst_isp']
available_cols = [c for c in geo_columns if c in enriched_flows.columns]

if available_cols:
    print("\n🗺️ Przykładowe dane geolokalizacyjne:")
    display(enriched_flows[available_cols].dropna().drop_duplicates().head(10))
else:
    print("Brak danych geolokalizacyjnych (sprawdź połączenie z API)")

---
## 📈 Krok 6: Wizualizacje

Generujemy wykresy i mapę Threat Intelligence.

In [ ]:
from visualizations import NetworkVisualizer
import matplotlib.pyplot as plt

# Inicjalizacja wizualizera
viz = NetworkVisualizer(output_dir="./output/visualizations")

# Słownik na ścieżki do wizualizacji
viz_paths = {}

In [ ]:
# Wykres rozkładu protokołów
viz_paths['protocols'] = viz.plot_protocol_distribution(flows)

# Wyświetl inline
from IPython.display import Image, display
display(Image(filename=viz_paths['protocols']))

In [ ]:
# Wykres alertów według severity
if not alerts_df.empty:
    viz_paths['severity'] = viz.plot_severity_distribution(alerts_df)
    display(Image(filename=viz_paths['severity']))
else:
    print("Brak alertów do wizualizacji")

In [ ]:
# Wykres alertów timeline
if not alerts_df.empty:
    viz_paths['alerts'] = viz.plot_alerts_timeline(alerts_df)
    display(Image(filename=viz_paths['alerts']))
else:
    print("Brak alertów do wizualizacji")

In [ ]:
# Macierz konfuzji
viz_paths['confusion'] = viz.plot_confusion_matrix(ml_metrics['confusion_matrix'])
display(Image(filename=viz_paths['confusion']))

In [ ]:
# Top IP
viz_paths['top_ips'] = viz.plot_top_ips(flows, ip_column='dst_ip', top_n=10)
display(Image(filename=viz_paths['top_ips']))

### 🗺️ Mapa Threat Intelligence

Interaktywna mapa pokazująca połączenia sieciowe z podziałem na normalne/podejrzane.

In [ ]:
# Mapa Threat Intelligence
map_path = viz.create_threat_intel_map(enriched_flows, alerts_df)

if map_path:
    print(f"✅ Mapa zapisana: {map_path}")
    print("\n🔗 Otwórz plik HTML w przeglądarce aby zobaczyć interaktywną mapę")
    
    # Próba wyświetlenia w notebook
    try:
        from IPython.display import IFrame
        display(IFrame(src=map_path, width=900, height=500))
    except:
        print(f"Otwórz ręcznie: {os.path.abspath(map_path)}")
else:
    print("⚠️ Nie udało się utworzyć mapy (brak danych geolokalizacyjnych lub folium)")

---
## 📄 Krok 7: Generowanie raportów

### 7a: Raport HTML

In [ ]:
from report_generator import ReportGenerator

# Generator raportów HTML
report_gen = ReportGenerator(output_dir="./output")

# Generowanie raportu
html_path = report_gen.generate_html_report(
    flows=flows,
    stats=stats,
    alerts=alerts_df if not alerts_df.empty else None,
    ml_metrics=ml_metrics,
    visualizations=viz_paths
)

print(f"✅ Raport HTML wygenerowany: {html_path}")
print(f"\n🔗 Otwórz w przeglądarce: {os.path.abspath(html_path)}")

### 7b: Raport LaTeX / PDF

In [ ]:
from latex_report_generator import LaTeXReportGenerator

# Generator raportów LaTeX
latex_gen = LaTeXReportGenerator(output_dir="./output")

# Generowanie raportu
tex_path = latex_gen.generate_latex_report(
    flows=flows,
    stats=stats,
    alerts=alerts_df if not alerts_df.empty else None,
    ml_metrics=ml_metrics,
    enriched_flows=enriched_flows,
    visualization_paths=viz_paths,
    title="Raport Analizy Ruchu Sieciowego",
    author="System Analizy Sieciowej",
    compile_pdf=True  # Ustaw False jeśli nie masz pdflatex
)

print(f"✅ Raport LaTeX wygenerowany: {tex_path}")

---
## ✅ Podsumowanie

Wszystkie kroki zakończone! Sprawdź wygenerowane pliki:

In [ ]:
print("=" * 60)
print("📁 WYGENEROWANE PLIKI")
print("=" * 60)

print("\n📂 data/")
for f in os.listdir("data"):
    size = os.path.getsize(f"data/{f}") / 1024
    print(f"   ├─ {f} ({size:.1f} KB)")

print("\n📂 output/")
for f in os.listdir("output"):
    path = f"output/{f}"
    if os.path.isdir(path):
        print(f"   ├─ {f}/")
        for sf in os.listdir(path):
            print(f"   │  └─ {sf}")
    else:
        size = os.path.getsize(path) / 1024
        print(f"   ├─ {f} ({size:.1f} KB)")

print("\n" + "=" * 60)
print("✅ WORKFLOW ZAKOŃCZONY POMYŚLNIE!")
print("=" * 60)

---
## 📋 Spełnione wymagania projektowe

In [ ]:
requirements = {
    "A.1 - Wczytywanie PCAP (NFStream/Scapy)": "✅",
    "A.2 - Statystyki przepływów": "✅",
    "D.1 - Reguły detekcyjne Python": "✅",
    "D.2 - Reguły Sigma": "✅",
    "ML.1 - Klasyfikacja ML": "✅",
    "ML.2 - Metryki FPR/TPR": "✅",
    "ML.3 - Trenowanie na nowych danych": "✅",
    "E.1 - Threat Intelligence enrichment": "✅",
    "V.1 - Wizualizacje alertów": "✅",
    "V.2 - Mapa geograficzna (nice-to-have)": "✅",
}

print("\n" + "=" * 50)
print("📋 WYMAGANIA PROJEKTU")
print("=" * 50)

for req, status in requirements.items():
    print(f"{status} {req}")

print("\n" + "=" * 50)
print("🎉 Wszystkie wymagania spełnione!")
print("=" * 50)